In [17]:
from pyiem.plot.use_agg import plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.colors as mpcolors
import datetime
import pytz
from backports.zoneinfo import ZoneInfo
from pandas.io.sql import read_sql
from pyiem.util import get_dbconn
cmap = cm.get_cmap("jet")
cmap.set_under("tan")
norm = mpcolors.BoundaryNorm(
    list(range(11)), 256
)


<ipython-input-17-5a827c7655c7>:11: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("jet"))
  cmap.set_under("tan")


In [18]:
dbconn = get_dbconn('iem')

df = read_sql("""SELECT valid, drct, id as station, vsby,
 wcht(tmpf::numeric, (sknt*1.15)::numeric), tmpf, greatest(sknt,gust),
 case when 'CLR' in (skyc1, skyc2, skyc3, skyc4) then 0 else 1 end as ovc,
 id from current_log c JOIN stations t ON (t.iemid = c.iemid)
 WHERE t.network in ('IA_ASOS', 'AWOS') and valid > '2020-12-07 00:00'
 and tmpf is not null 
 ORDER by valid ASC
 """, dbconn, index_col=None)

In [3]:
x = []
y = []
base = datetime.datetime(2020, 10, 22, 10).replace(tzinfo=pytz.utc)
for hr in range(24):
    sts = base + datetime.timedelta(hours=hr)
    ets = sts + datetime.timedelta(hours=1)
    x.append(sts)
    df2 = df[(df['valid'] >= sts) & (df['valid'] < ets)]
    df2 = df2.sort_values('tmpf')
    y.append(df2['tmpf'].max() - df2['tmpf'].min())
    print("%s %s %s %s" % (hr, df2[['id', 'tmpf', 'valid']].head(1), df2[['id', 'tmpf', 'valid']].tail(1), y[-1]))


0       id  tmpf                     valid
966  EST  30.9 2020-10-22 10:52:00+00:00       id  tmpf                     valid
896  EOK  50.0 2020-10-22 10:15:00+00:00 19.1
1        id  tmpf                     valid
1129  EST  30.9 2020-10-22 11:52:00+00:00        id  tmpf                     valid
1160  EOK  51.8 2020-10-22 11:55:00+00:00 20.9
2        id  tmpf                     valid
1295  EST  30.2 2020-10-22 12:49:00+00:00        id  tmpf                     valid
1269  FSW  51.8 2020-10-22 12:35:00+00:00 21.599999999999998
3        id  tmpf                     valid
1465  EST  30.2 2020-10-22 13:49:00+00:00        id  tmpf                     valid
1503  ICL  57.2 2020-10-22 13:55:00+00:00 27.000000000000004
4        id  tmpf                     valid
1639  EST  30.9 2020-10-22 14:52:00+00:00        id  tmpf                     valid
1664  EOK  60.8 2020-10-22 14:55:00+00:00 29.9
5        id  tmpf                     valid
1697  EST  30.9 2020-10-22 15:04:00+00:00        id  tmpf

In [24]:
sts = datetime.datetime(2020, 12, 7, 0, tzinfo=ZoneInfo("America/Chicago"))
ets = datetime.datetime(2020, 12, 8, 0, tzinfo=ZoneInfo("America/Chicago"))
interval = datetime.timedelta(hours=1)
xticks = []
xticklabels = []
now = sts
while now <= ets:
    if now.hour % 6 != 0:
        now += interval
        continue
    fmt = "%-I %p" if now.hour != 0 else '%-I %p\n%-d %b'
    xticks.append(now)
    xticklabels.append(now.strftime(fmt))
    now += interval

(fig, axes) = plt.subplots(1, 1, figsize=(12, 6.75))
ax = axes
#df2 = df[df['ovc'] == 1]
#ax.scatter(df2['valid'].values, df2['tmpf'].values, marker='x', color='grey', alpha=0.2,
#          label='Cloudy')
df2 = df[df['ovc'] == 1]
ax.scatter(df2['valid'].values, df2['tmpf'].values, marker='+', alpha=1, label="Cloudy")
df2 = df[df['ovc'] != 1]
ax.scatter(df2['valid'].values, df2['tmpf'].values, marker='o', alpha=1, label="Clear")
ax.set_title("7 December 2020 Iowa Airport Observations", fontsize=24)
ax.set_ylabel("Air Temperature $^\circ$F", fontsize=18)
ax.legend(ncol=2, loc=4)
#ax.set_yticks([0, 45, 90, 135, 180, 225, 270, 315, 360])
#ax.set_yticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N'], fontsize=14)
#ax.set_ylim(-1, 361)
#leg = ax.legend()
#for lh in leg.legendHandles: 
#    lh.set_alpha(1)
ax.grid(True)
ax.set_ylim(10, 55)

"""
x = []
y = []
for dt, gdf in df.groupby(df['valid'].dt.strftime("%Y%m%d%H")):
    rng = gdf['tmpf'].max() - gdf['tmpf'].min()
    if rng < 10:
        continue
    x.append(datetime.datetime.strptime(dt, '%Y%m%d%H'))
    y.append(rng)

ax2 = ax.twinx()
ax2.plot(x, y, color='white', zorder=5, lw=4)
ax2.plot(x, y, color='r', zorder=5, lw=3)
ax2.set_xticks([])
ax2.set_ylim(0, 60)
ax2.tick_params(axis='y', colors='red')
ax2.set_ylabel("Air Temperature Range $^\circ$F", color='r')
"""
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, fontsize=18)
ax.set_xlim(sts, ets)


#ax = axes[1]
#ax.plot(x, y)
#ax.grid(True)
#ax.set_ylabel("Temperature Spread $^\circ$F")
fig.savefig('201208.png')
plt.close()